In [1]:
# Jupyter Notebook: Simulation of Parlay Smart Contract LP 

# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from collections import defaultdict

# Set display options
%matplotlib inline


Introduction

This is a simulation of our smart contract LP. It reads the odds of whitelisted prediction market and lets users place parlays from 2-10 legs. The payout is inline with the odds of all events occurring with a set margin. This simulation will show the feasibility and profitability of this system.

Rules:

    The balance of the contract may never be negative, wagers must be refused if it is not possible to payout the winner
    Bets cannot be 'cashed out' early and only close with a win or loss
    All prediction markets resolve as either YES / NO. Invalid markets and multiple options are not considered
    All events are settled at the same time
    1000 parlays are placed each betting cycle, after the cycle winners are paid out and new markets whitelisted and the cycle starts again

Contract Parameters:

    The contract starts with $10000 of liquidity
    The reserves required for taking a single bet must not exceed 10% of the available liquidity
    Hedging wagers will be placed to reduce the amount of reserves required
    Reserves may be shared between bets in which the winner cannot possibly win in both scenarios
    A flat margin will be taken on all payouts to ensure a profit. Starts at 5% but subject to change
    There are a set of 10 unique events to make parlays on, subject to change

Bettors:

    Bets are made in random sizes
    Bets are made with random number of legs 
    Prediction market odds are not static, but normally distributed around 50%, parlay odds are correlated to calculated odds and number of legs





1. Setting Up the Simulation
1.1 Initialize Contract Parameters

In [2]:
# Contract parameters
starting_liquidity = 10000.0  # Starting liquidity in dollars
liquidity = starting_liquidity  # Current liquidity
max_exposure_per_bet = 0.10  # 10% of available liquidity
flat_margin = 0.05  # 5% margin
num_events = 10  # Number of unique events
events = [f'Event_{i+1}' for i in range(num_events)]  # List of event names
betting_cycles = 5  # Number of betting cycles to simulate
odds_limit = 1000.0  # Maximum allowed odds (1000 to 1)


1.2 Define Bet

In [3]:
# Bettor parameters
num_bettors_per_cycle = 1000  # Number of bettors per betting cycle
min_bet_size = 1.0  # Minimum bet size
max_bet_size = 100.0  # Maximum bet size
min_legs = 2  # Minimum number of legs in a parlay
max_legs = 10  # Maximum number of legs in a parlay


1.3 Initialize Data Structures

In [4]:
# Data structures to store simulation results
liquidity_over_time = [liquidity]
profit_over_time = []


2. Simulating Betting Cycles

We will simulate a specified number of betting cycles. In each cycle:

    Simulate Bettor Behavior: Bettors place bets with random sizes and random legs. Event odds are generated dynamically for each bettor.
    Enforce Contract Rules: Bets exceeding liquidity constraints are refused.
    Calculate Reserves and Hedging: Determine the reserves required and apply hedging strategies, accounting for partial offsets due to different odds.
    Settle Bets: Simulate the outcomes and settle bets.
    Update Liquidity: Adjust the contract's liquidity based on payouts and profits.

2.1 Function to Simulate a Single Bet with Dynamic Odds

In [5]:
def simulate_single_bet():
    """
    Simulate a single bettor placing a parlay bet with dynamic odds.
    Returns the bet dictionary or None if the bet is rejected.
    """
    bet_size = np.random.uniform(min_bet_size, max_bet_size)
    num_legs = np.random.randint(min_legs, max_legs + 1)
    selected_events = np.random.choice(events, size=num_legs, replace=False)
    selected_outcomes = np.random.choice([0, 1], size=num_legs)  # 0: NO, 1: YES

    # Generate odds for the selected events at the time of bet placement
    selected_odds = []
    for event in selected_events:
        probability = np.clip(np.random.normal(0.5, 0.15), 0.1, 0.9)  # Probabilities between 0.1 and 0.9
        decimal_odds = 1 / probability  # Convert probability to decimal odds
        selected_odds.append(decimal_odds)

    # Calculate total odds for the parlay
    parlay_odds = np.prod(selected_odds)

    # Apply the margin
    parlay_odds_with_margin = parlay_odds / (1 + flat_margin)

    # Check if parlay odds exceed the odds limit
    if parlay_odds_with_margin > odds_limit:
        return None  # Bet is rejected due to high odds

    bet = {
        'bet_size': bet_size,
        'num_legs': num_legs,
        'selected_events': selected_events,
        'selected_outcomes': selected_outcomes,
        'selected_odds': selected_odds,
        'parlay_odds': parlay_odds_with_margin,
        'potential_payout': bet_size * parlay_odds_with_margin
    }
    return bet


Explanation:

    Dynamic Odds: For each bettor, we generate odds for their selected events at the time of bet placement. This simulates changing odds in the prediction markets.
    Event Odds Generation: The odds are generated based on probabilities normally distributed around 50%, ensuring they fall between 10% and 90%.

## 2.2 Function to Determine if a Bet Can Be Accepted 

To consider shared reserves due to mutually exclusive bets, we need to calculate the maximum possible payout across all possible outcomes with the existing bets and the new bet. If adding the new bet doesn't increase the maximum possible payout beyond the liquidity, we can accept it.

Due to computational limitations, we will:

    Generate all possible outcome combinations of the events involved in the accepted bets and the new bet.
    Calculate the total potential payout for each outcome.
    Determine the maximum total payout.
    Compare it to the liquidity.

Note: Since the number of possible outcomes increases exponentially with the number of events, we need to limit the number of events considered.

To make it computationally feasible, we will:

    Limit the number of events considered to a manageable size, e.g., events involved in the current bets (usually less than or equal to 20).
    Use an efficient data structure to store and calculate payouts.

In [6]:
# Precompute all possible outcomes for a given number of events
from functools import lru_cache

@lru_cache(maxsize=None)
def get_all_possible_outcomes(num_events):
    return list(itertools.product([0, 1], repeat=num_events))


In [7]:
def can_accept_bet_with_shared_reserves(accepted_bets, new_bet, current_liquidity):
    """Optimized shared reserves check with early exits"""
    
    # Early exit 1: If no existing bets, just check new bet's payout
    if not accepted_bets:
        return new_bet['potential_payout'] <= current_liquidity
        
    # Early exit 2: Check if just the new bet exceeds liquidity
    if new_bet['potential_payout'] > current_liquidity:
        return False
    
    # Early exit 3: Quick sum check - if total of all potential payouts is less than liquidity
    total_potential = sum(bet['potential_payout'] for bet in accepted_bets) + new_bet['potential_payout']
    if total_potential <= current_liquidity:
        return True
        
    # Combine events from accepted bets and the new bet
    events_in_bets = set()
    for bet in accepted_bets + [new_bet]:
        events_in_bets.update(bet['selected_events'])
    events_in_bets = list(events_in_bets)
    
    # Early exit 4: Too many events to compute combinations
    if len(events_in_bets) > 15:
        return False
    
    # Create lookup for faster event checking
    event_indices = {event: idx for idx, event in enumerate(events_in_bets)}
    
    # Get cached outcomes
    all_possible_outcomes = get_all_possible_outcomes(len(events_in_bets))
    
    # Check each outcome until we find one that exceeds liquidity
    for outcome in all_possible_outcomes:
        total_payout = 0.0
        
        # Check each bet against this outcome
        for bet in accepted_bets + [new_bet]:
            # Quick check if bet wins
            bet_wins = True
            for event, desired_outcome in zip(bet['selected_events'], bet['selected_outcomes']):
                if outcome[event_indices[event]] != desired_outcome:
                    bet_wins = False
                    break
                    
            if bet_wins:
                total_payout += bet['potential_payout']
                # Early exit 5: Stop checking this outcome if already over liquidity
                if total_payout > current_liquidity:
                    return False
    
    return True

## 2.3 We hedge at the portfolio level to 

In [8]:
def calculate_net_exposure(accepted_bets, current_hedges):
    """
    Calculate net exposure per event considering both bets and existing hedges.
    """
    event_exposure = defaultdict(lambda: {'yes': 0, 'no': 0, 'net': 0})
    
    # Calculate exposure from parlays
    for bet in accepted_bets:
        for i, (event, outcome) in enumerate(zip(bet['selected_events'], bet['selected_outcomes'])):
            # Calculate probability of other events in parlay
            other_prob = 1.0
            for j, odds in enumerate(bet['selected_odds']):
                if j != i:
                    other_prob *= 1/odds
            
            exposure = bet['potential_payout'] * other_prob
            if outcome == 1:
                event_exposure[event]['yes'] += exposure
            else:
                event_exposure[event]['no'] += exposure
    
    # Subtract existing hedge exposure
    for hedge in current_hedges:
        if hedge['side'] == 'yes':
            event_exposure[hedge['event']]['yes'] -= hedge['size'] * hedge['odds']
        else:
            event_exposure[hedge['event']]['no'] -= hedge['size'] * hedge['odds']
    
    # Calculate net exposure
    for event in event_exposure:
        event_exposure[event]['net'] = event_exposure[event]['yes'] - event_exposure[event]['no']
    
    return dict(event_exposure)

In [9]:
def get_current_market_odds(event, side):
    """
    Simulate getting current market odds for an event.
    In real implementation, this would query the prediction market.
    """
    base_probability = np.clip(np.random.normal(0.5, 0.15), 0.1, 0.9)
    
    # Return appropriate odds based on side
    if side == 'yes':
        return 1/base_probability
    else:
        return 1/(1-base_probability)

In [10]:
def try_place_hedge(event, side, size, available_liquidity):
    """
    Attempt to place a hedge at current market odds.
    Returns (success, hedge_details, remaining_liquidity)
    """
    # Get current market odds at execution time
    execution_odds = get_current_market_odds(event, side)
    
    # Check if we can still afford this hedge at current odds
    cost = size
    potential_return = size * execution_odds
    
    if cost <= available_liquidity:
        hedge = {
            'event': event,
            'side': side,
            'size': size,
            'odds': execution_odds,  # Locked in odds at execution
        }
        return True, hedge, available_liquidity - cost
    
    return False, None, available_liquidity

In [11]:
def update_hedge_positions(accepted_bets, current_hedges, current_liquidity):
    """
    Update hedge positions based on current net exposure.
    Returns new hedges and remaining liquidity.
    """
    exposure = calculate_net_exposure(accepted_bets, current_hedges)
    hedge_threshold = current_liquidity * 0.05  # Hedge when exposure > 15% of liquidity
    max_hedge_per_event = current_liquidity * 0.10  # Max 10% of liquidity per hedge
    new_hedges = current_hedges.copy()
    remaining_liquidity = current_liquidity
    
    for event, exp in exposure.items():
        if abs(exp['net']) > hedge_threshold:
            # Close opposing positions first
            new_hedges = [h for h in new_hedges if h['event'] != event]
            
            # Calculate new hedge size
            hedge_size = min(abs(exp['net']) * 0.8, max_hedge_per_event)
            
            if hedge_size > 0 and remaining_liquidity >= hedge_size:
                # Get current market price for hedge
                probability = np.clip(np.random.normal(0.5, 0.15), 0.1, 0.9)
                
                new_hedges.append({
                    'event': event,
                    'side': 'no' if exp['net'] > 0 else 'yes',
                    'size': hedge_size,
                    'odds': 1/probability if exp['net'] < 0 else 1/(1-probability)
                })
                remaining_liquidity -= hedge_size
    
    return new_hedges, remaining_liquidity

In [12]:
def settle_hedges(hedges, event_outcomes):
    """
    Calculate returns from hedge positions based on event outcomes.
    """
    total_hedge_return = 0.0
    for hedge in hedges:
        if (hedge['side'] == 'yes' and event_outcomes[hedge['event']] == 1) or \
           (hedge['side'] == 'no' and event_outcomes[hedge['event']] == 0):
            total_hedge_return += hedge['size'] * hedge['odds']
    return total_hedge_return

Explanation:

    Net Exposure Calculation: We calculate the net exposure for each event by finding the difference in potential payouts between the two outcomes.
    Hedging Bets: The contract places hedging bets on the prediction market to offset the net exposure.
    Hedging Cost: The cost of hedging bets is accounted for, and we ensure it does not exceed liquidity.
    Reserves Required: After hedging, the net exposure (potential payouts to bettors) is reduced, and the reserves required are updated accordingly.

2.4 Functions to Simulate Event Outcomes and Settle Bets



In [13]:
def simulate_event_outcomes(events):
    """
    Simulate the outcomes of all events (0: NO, 1: YES).
    """
    outcomes = np.random.choice([0, 1], size=len(events))
    return dict(zip(events, outcomes))
    

Explanation:

    Settling Bets: Calculate the total payout to bettors based on the outcomes.
    Settling Hedging Bets: Calculate the returns from hedging bets based on event outcomes.

Explanation:

    Dynamic Odds and Partial Hedging: The simulation now accounts for dynamic odds and partial hedging when calculating reserves.
    Adjusting Accepted Bets: If total reserves required exceed available liquidity, the simulation removes bets with the highest potential payout until the reserves required are within liquidity constraints.
    Liquidity Updates: The liquidity is updated after each cycle based on the actual bets accepted and the outcomes.

In [14]:
def run_simulation(run_number, enable_hedging=False):
    """
    Run simulation with optional hedging.
    
    Parameters:
        run_number: int - The simulation run number
        enable_hedging: bool - Whether to enable dynamic hedging
    """
    liquidity = starting_liquidity
    liquidity_over_time = [liquidity]
    profit_over_time = []
    hedge_returns_over_time = []
    metrics = {
        'total_bets_accepted': 0,
        'total_wagers': 0,
        'total_hedge_cost': 0,
        'total_hedge_returns': 0,
        'max_drawdown': 0,
        'min_liquidity': liquidity,
    }
    
    for cycle in range(betting_cycles):
        accepted_bets = []
        current_hedges = []
        total_wagers = 0.0
        total_payout_to_bettors = 0.0
        available_liquidity = liquidity
        
        for _ in range(num_bettors_per_cycle):
            bet = simulate_single_bet()
            if bet is None:
                continue
                
            required_reserve = bet['potential_payout'] - bet['bet_size']
            max_allowed_reserve = available_liquidity * max_exposure_per_bet
            
            if required_reserve > max_allowed_reserve:
                continue
                
            if not can_accept_bet_with_shared_reserves(accepted_bets, bet, available_liquidity):
                continue
                
            accepted_bets.append(bet)
            total_wagers += bet['bet_size']
            metrics['total_bets_accepted'] += 1
            
            # Update hedges only if enabled
            if enable_hedging:
                current_hedges, available_liquidity = update_hedge_positions(
                    accepted_bets, current_hedges, available_liquidity
                )
        
        if not accepted_bets:
            profit_over_time.append(0.0)
            hedge_returns_over_time.append(0.0)
            liquidity_over_time.append(liquidity)
            continue
            
        metrics['total_wagers'] += total_wagers
        
        # Simulate outcomes and settle
        event_outcomes = simulate_event_outcomes(events)
        
        # Calculate bettor payouts
        for bet in accepted_bets:
            bet_wins = True
            for event, outcome in zip(bet['selected_events'], bet['selected_outcomes']):
                if event_outcomes[event] != outcome:
                    bet_wins = False
                    break
            if bet_wins:
                total_payout_to_bettors += bet['potential_payout']
        
        # Calculate hedge returns if enabled
        hedge_returns = 0.0
        if enable_hedging:
            hedge_returns = settle_hedges(current_hedges, event_outcomes)
            metrics['total_hedge_returns'] += hedge_returns
            if current_hedges:
                metrics['total_hedge_cost'] += sum(h['size'] for h in current_hedges)
        
        hedge_returns_over_time.append(hedge_returns)
        
        # Update liquidity and track metrics
        profit = total_wagers - total_payout_to_bettors + hedge_returns
        liquidity += profit
        
        metrics['min_liquidity'] = min(metrics['min_liquidity'], liquidity)
        current_drawdown = starting_liquidity - liquidity
        metrics['max_drawdown'] = max(metrics['max_drawdown'], current_drawdown)
        
        liquidity_over_time.append(liquidity)
        profit_over_time.append(profit)
        
        if run_number == 1:  # Only print for first run to reduce output
            print(f"Cycle {cycle + 1} Results:")
            print(f"Total Wagers: ${total_wagers:.2f}")
            print(f"Bets Accepted: {len(accepted_bets)}")
            if enable_hedging:
                print(f"Active Hedges: {len(current_hedges)}")
                print(f"Hedge Returns: ${hedge_returns:.2f}")
            print(f"Cycle Profit: ${profit:.2f}")
            print(f"Current Liquidity: ${liquidity:.2f}\n")
    
    return {
        'run_number': run_number,
        'hedging_enabled': enable_hedging,
        'liquidity_over_time': liquidity_over_time,
        'profit_over_time': profit_over_time,
        'hedge_returns_over_time': hedge_returns_over_time,
        'ending_liquidity': liquidity,
        'metrics': metrics
    }

4.3 Summary Statistics

In [15]:
def plot_simulation_comparisons(hedged_results, unhedged_results):
    """Plot and save comprehensive comparison between hedged and unhedged simulations"""
    
    # 1. Liquidity Over Time
    plt.figure(figsize=(12, 6))
    for result in unhedged_results:
        plt.plot(result['liquidity_over_time'], alpha=0.3, color='red')
    for result in hedged_results:
        plt.plot(result['liquidity_over_time'], alpha=0.3, color='blue')
    avg_liquidity_unhedged = np.mean([result['liquidity_over_time'] for result in unhedged_results], axis=0)
    avg_liquidity_hedged = np.mean([result['liquidity_over_time'] for result in hedged_results], axis=0)
    plt.plot(avg_liquidity_unhedged, color='red', linewidth=2, label='Average Unhedged')
    plt.plot(avg_liquidity_hedged, color='blue', linewidth=2, label='Average Hedged')
    plt.title('Liquidity Over Time for Multiple Simulation Runs')
    plt.xlabel('Betting Cycle')
    plt.ylabel('Liquidity ($)')
    plt.grid(True)
    plt.legend()
    plt.savefig('liquidity_over_time.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 2. Cumulative Returns
    profits_matrix_unhedged = np.array([result['profit_over_time'] for result in unhedged_results])
    profits_matrix_hedged = np.array([result['profit_over_time'] for result in hedged_results])
    
    cumulative_profits_matrix_unhedged = np.cumsum(profits_matrix_unhedged, axis=1)
    cumulative_profits_matrix_hedged = np.cumsum(profits_matrix_hedged, axis=1)
    
    avg_cumulative_profits_unhedged = np.mean(cumulative_profits_matrix_unhedged, axis=0)
    avg_cumulative_profits_hedged = np.mean(cumulative_profits_matrix_hedged, axis=0)
    
    min_cumulative_profits_unhedged = np.min(cumulative_profits_matrix_unhedged, axis=0)
    max_cumulative_profits_unhedged = np.max(cumulative_profits_matrix_unhedged, axis=0)
    min_cumulative_profits_hedged = np.min(cumulative_profits_matrix_hedged, axis=0)
    max_cumulative_profits_hedged = np.max(cumulative_profits_matrix_hedged, axis=0)

    plt.figure(figsize=(12, 6))
    cycles = range(1, betting_cycles + 1)
    plt.plot(cycles, avg_cumulative_profits_unhedged, label='Average Unhedged Returns', color='red')
    plt.plot(cycles, avg_cumulative_profits_hedged, label='Average Hedged Returns', color='blue')
    plt.fill_between(cycles, min_cumulative_profits_unhedged, max_cumulative_profits_unhedged, 
                     color='red', alpha=0.2, label='Unhedged Range')
    plt.fill_between(cycles, min_cumulative_profits_hedged, max_cumulative_profits_hedged, 
                     color='blue', alpha=0.2, label='Hedged Range')
    plt.title('Cumulative Returns Comparison')
    plt.xlabel('Betting Cycle')
    plt.ylabel('Cumulative Returns ($)')
    plt.legend()
    plt.grid(True)
    plt.savefig('cumulative_returns.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 3. Standard Deviation Over Time
    liquidity_matrix_unhedged = np.array([result['liquidity_over_time'] for result in unhedged_results])
    liquidity_matrix_hedged = np.array([result['liquidity_over_time'] for result in hedged_results])
    
    liquidity_std_dev_unhedged = np.std(liquidity_matrix_unhedged, axis=0)
    liquidity_std_dev_hedged = np.std(liquidity_matrix_hedged, axis=0)

    plt.figure(figsize=(12, 6))
    cycles = range(betting_cycles + 1)
    plt.plot(cycles, liquidity_std_dev_unhedged, label='Unhedged Std Dev', color='red')
    plt.plot(cycles, liquidity_std_dev_hedged, label='Hedged Std Dev', color='blue')
    plt.title('Standard Deviation of Liquidity Over Time')
    plt.xlabel('Betting Cycle')
    plt.ylabel('Standard Deviation ($)')
    plt.legend()
    plt.grid(True)
    plt.savefig('standard_deviation.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 4. Mean and Median Liquidity
    mean_liquidity_unhedged = np.mean(liquidity_matrix_unhedged, axis=0)
    median_liquidity_unhedged = np.median(liquidity_matrix_unhedged, axis=0)
    mean_liquidity_hedged = np.mean(liquidity_matrix_hedged, axis=0)
    median_liquidity_hedged = np.median(liquidity_matrix_hedged, axis=0)

    plt.figure(figsize=(12, 6))
    plt.plot(cycles, mean_liquidity_unhedged, label='Mean Unhedged', color='red')
    plt.plot(cycles, median_liquidity_unhedged, label='Median Unhedged', color='red', linestyle='--')
    plt.plot(cycles, mean_liquidity_hedged, label='Mean Hedged', color='blue')
    plt.plot(cycles, median_liquidity_hedged, label='Median Hedged', color='blue', linestyle='--')
    plt.title('Mean and Median Liquidity Comparison')
    plt.xlabel('Betting Cycle')
    plt.ylabel('Liquidity ($)')
    plt.legend()
    plt.grid(True)
    plt.savefig('mean_median_liquidity.png', dpi=300, bbox_inches='tight')
    plt.close()

    # 5. Profit Distribution
    mean_profit_unhedged = np.mean(profits_matrix_unhedged, axis=0)
    std_profit_unhedged = np.std(profits_matrix_unhedged, axis=0)
    mean_profit_hedged = np.mean(profits_matrix_hedged, axis=0)
    std_profit_hedged = np.std(profits_matrix_hedged, axis=0)

    plt.figure(figsize=(12, 6))
    cycles = range(1, betting_cycles + 1)
    plt.plot(cycles, mean_profit_unhedged, label='Mean Unhedged Profit', color='red')
    plt.plot(cycles, mean_profit_hedged, label='Mean Hedged Profit', color='blue')
    plt.fill_between(cycles, 
                     mean_profit_unhedged - std_profit_unhedged,
                     mean_profit_unhedged + std_profit_unhedged,
                     color='red', alpha=0.2, label='Unhedged Std Dev')
    plt.fill_between(cycles,
                     mean_profit_hedged - std_profit_hedged,
                     mean_profit_hedged + std_profit_hedged,
                     color='blue', alpha=0.2, label='Hedged Std Dev')
    plt.title('Mean Profit per Cycle with Standard Deviation')
    plt.xlabel('Betting Cycle')
    plt.ylabel('Profit ($)')
    plt.legend()
    plt.grid(True)
    plt.savefig('profit_distribution.png', dpi=300, bbox_inches='tight')
    plt.close()

    # Save summary statistics to text file
    with open('simulation_summary.txt', 'w') as f:
        f.write("Summary Statistics:\n\n")
        f.write("Unhedged Results:\n")
        f.write(f"Average Ending Liquidity: ${np.mean([r['ending_liquidity'] for r in unhedged_results]):.2f}\n")
        f.write(f"Final Std Dev: ${np.std([r['ending_liquidity'] for r in unhedged_results]):.2f}\n")
        f.write(f"Total Profit: ${np.mean([sum(r['profit_over_time']) for r in unhedged_results]):.2f}\n\n")
        
        f.write("Hedged Results:\n")
        f.write(f"Average Ending Liquidity: ${np.mean([r['ending_liquidity'] for r in hedged_results]):.2f}\n")
        f.write(f"Final Std Dev: ${np.std([r['ending_liquidity'] for r in hedged_results]):.2f}\n")
        f.write(f"Total Profit: ${np.mean([sum(r['profit_over_time']) for r in hedged_results]):.2f}\n")

In [16]:
# Run simulations
num_simulation_runs = 5
hedged_results = []
unhedged_results = []

In [17]:
# Run both types of simulations
for run in range(num_simulation_runs):
    print(f"Running simulation {run + 1}")
    hedged_result = run_simulation(run + 1, enable_hedging=True)
    unhedged_result = run_simulation(run + 1, enable_hedging=False)
    hedged_results.append(hedged_result)
    unhedged_results.append(unhedged_result)



Running simulation 1
Cycle 1 Results:
Total Wagers: $869.75
Bets Accepted: 24
Active Hedges: 4
Hedge Returns: $1691.28
Cycle Profit: $2457.38
Current Liquidity: $12457.38

Cycle 2 Results:
Total Wagers: $1024.22
Bets Accepted: 25
Active Hedges: 4
Hedge Returns: $3787.46
Cycle Profit: $3386.02
Current Liquidity: $15843.40

Cycle 3 Results:
Total Wagers: $1512.39
Bets Accepted: 28
Active Hedges: 6
Hedge Returns: $6654.11
Cycle Profit: $7973.35
Current Liquidity: $23816.75

Cycle 4 Results:
Total Wagers: $5100.59
Bets Accepted: 116
Active Hedges: 4
Hedge Returns: $5464.34
Cycle Profit: $8333.19
Current Liquidity: $32149.95

Cycle 5 Results:
Total Wagers: $14171.67
Bets Accepted: 324
Active Hedges: 1
Hedge Returns: $0.00
Cycle Profit: $-2082.07
Current Liquidity: $30067.88

Cycle 1 Results:
Total Wagers: $7176.06
Bets Accepted: 197
Cycle Profit: $-483.52
Current Liquidity: $9516.48

Cycle 2 Results:
Total Wagers: $7541.23
Bets Accepted: 212
Cycle Profit: $1529.26
Current Liquidity: $11045.

In [19]:
# Plot comparisons
plot_simulation_comparisons(hedged_results, unhedged_results)